# PyTorch - CIFAR-10

In [8]:
import torch
import torchvision
import numpy as np
import pickle
import os

## Download Dataset

In [ ]:
# Download the dataset
!wget -O cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

# Extract the dataset
!tar -xvzf cifar-10-python.tar.gz

# Remove the tar file
!rm cifar-10-python.tar.gz

## Create PyTorch Dataset

In [23]:
class CIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        
        # Load the data batches
        self.data = []
        self.labels = []
        for i in range(1, 6):
            filename = os.path.join(self.root_dir, f"data_batch_{i}")
            with open(filename, 'rb') as fo:
                batch = pickle.load(fo, encoding='bytes')
            self.data.append(batch[b'data'])
            self.labels += batch[b'labels']

        self.data = np.concatenate(self.data, axis=0)
        self.data = self.data.reshape((50000, 3, 32, 32))
        self.data = self.data.transpose((0, 2, 3, 1))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        
        if self.transform:
            img = self.transform(img)
            
        return img, label


### Dataset Normalization

In [24]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

BATCH_SIZE = 64

train_dataset = CIFAR10Dataset(root_dir='cifar-10-batches-py', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Define Network

In [34]:
import torch.nn as nn
import torch.optim as optim

# Define the CNN model
class CIFAR10Model(nn.Module):
    def __init__(self):
        super(CIFAR10Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Define Loss Function and Optimizer

In [35]:
# Create the model, loss function, and optimizer
model = CIFAR10Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Train The Model

In [37]:
# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step(batch_size=BATCH_SIZE)
        
        running_loss += loss.item()
        if i % 200 == 199:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200}")
            running_loss = 0.0

ValueError: Expected input batch_size (256) to match target batch_size (64).